In [8]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [9]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [10]:
# A class to represent a Webpage
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        print('The response is:\n ', response)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        #print('Soup body:/n ', soup.body)
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [12]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [13]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
import ollama
def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

In [16]:
summarize("https://www.playstation.com/en-us")

The response is:
  <Response [200]>


"# PlayStation Official Site: Consoles, Games, Accessories & More\n\n## Overview\n\nThe PlayStation official website offers a wide range of gaming consoles, games, accessories, and more. The site is divided into various sections, including new releases, upcoming games, PlayStation Plus benefits, and exclusive deals.\n\n## News and Announcements\n\n*   Call of Duty: Black Ops 7 launches on November 14th.\n*   Ghost of Yōtei Legends DLC is coming in 2026.\n*   Lords of the Fallen II unveiled, releases 2026 on PS5.\n*   The Blood of Dawnwalker — a story-driven, open-world RPG set in 14th-century Europe, announced.\n*   Resident Evil Requiem, Pragmata, and Onimusha: Way of the Sword announced.\n\n## Games\n\nThe website features a vast library of games, including:\n\n*   **PS5 Exclusive Titles**\n    *   Horizon Forbidden West\n    *   Gran Turismo 7\n    *   Marvel's Spider Man Miles Morales\n    *   Ratchet & Clank: Rift Apart\n*   **PS4 and PS5 Games**\n    *   Death Stranding Director'